Machine learning and microscopy
=====================

Moving to machine learning
------------

* Machine learning at heart is a software toolset to apply statistical methods to find patterns in data.
* “The ability [of machine learning] to automatically identify patterns in data [...] is particularly important when the expert knowledge is incomplete or inaccurate, when the amount of available data is too large to be handled manually, or when there are exceptions to the general cases” [1]

![ML](images/machine_learning_joke.jpg)

Why NOT use maching learning?
------------

* It is very tempting to start applying machine learning approaches to everything, but it is critical to step back and see if it makes sense.
* Some major reasons NOT to apply an machine learning approach would be:
    * There are existing methods that do the job well - there is no point inventing a square wheel.
    * You don't have enough data - in general you want to have a ratio of 10:1 between *datapoints* and *features*.
    * Your data is poorly organized or heavily biased - if you are not sure about the history and compatibility of your data you will not be able to trust any of your predictions.

[1]: Yip KY, Cheng C, Gerstein M. Machine learning and genome annotation: a match meant to be? Genome Biol. 2013;14(5):205.

Recent examples
====================

Image segmentation
---------------------

* A very common application of machine learning in microscopy is image segmentation i.e. automatically detecting features in images.
* This is generally achieved using Convolutional Neural Networks (CNN) and the approach is the backbone for all the animal classification methods you have likely seen already.
* This classification is an example of *supervised learning*, where the training data is labelled by the user. The algorithm then learns this classification and is able to make accurate predictions on new data.
* For an example, we often use the U-Net method [1], which is a powerful approach, widely used in many CNN applications. For biomedial imaging, they actually broke some of our inital rules...they had very little data, but used extensive *augmentation* to expand the valid data set. In particular (and general to microscopy of soft systems) they introduced:
    * shift and rotation invariance 
    * gray value variations
    * random elastic deformations
* It is not perfect (77%), but much better than any alternatives at that time.

![segment](images/segment.png)

[1]: Ronneberger, O., Fischer, P. & Brox, T. U-Net: Convolutional Networks for Biomedical Image Segmentation. arXiv:1505.04597 [cs] (2015).



Cell classification
---------------------

* In terms of searching for patterns in images that demonstrate a correlation between a measurement and a hypothesis, machine learning image analysis can also be very powerful.
* In the example, they looked at multi-channel AFM data of healthy and cancerous (fixed) cells [1].

![cancer_cells](images/cancer_cells.png)

* Model was trained on the imaging channels independently and in combination, and the resultant classification accuracy was studied as a function of measured parameters. Very high accuracy was achieved using a combination of channels.

![cancer_acc](images/cancer_acc.png)

[1]: Prasad, S. et al. Atomic Force Microscopy Detects the Difference in Cancer Cells of Different Neoplastic Aggressiveness via Machine Learning. Advanced NanoBiomed Research 1, 2000116 (2021).


Active learning
--------------------

* A powerful development in experimental design is the introduction of active learning (also referred to as *Bayesian Optimisation*), where measurements are dynamically chosen to optimise the information obtained.
* In piezoresponse force microscopy, this has been used to automate the measurement of spectra at regions of specific interest [1]. The system is first trained on known surface features and spectra.

![active_model](images/active_model.png)

* Then, when it is linked to the driving software of an SPM, the network itself can make a decision about the most useful place to measure the spectra. These *decisions* can then actually tell us something about the correlation between image features and the nature of the spectra.

![active_image](images/active_image.png)

[1]: Liu, Y., Kelley, K.P., Vasudevan, R.K. et al. Experimental discovery of structure–property relationships in ferroelectric materials via active learning. Nat Mach Intell 4, 341–350 (2022).

Machine learning methods
============================

* There are a wide variety of machine learning approaches and as a general rule it is better to use the simplest one that works. In microscopy, since we are often working with images, then Neural Networks (NN) are an obvious place to start, as there has been an enormous effort in recent years to optimise NN tools for image analysis. They also play role in all the examples/tutorials we are considering. Do NOT use them by default...

Neural Networks
-------------------

* The starting point of a neural network is the neuron itself (called a *perceptron* in the context of machine learning) - it takes inputs, applies some operations to them and produces an output [1].

![neuron](images/perceptron.svg)

* First each input $x_1,x_2$ is multiplied by weights $w_1,w_2$. Then all the weighted inputs are added together with a bias $b$ and an activation function $f$ is applied to the total:

$y = f((w_1*x_1)+(w_2*x_2) + b$)

* The bias reflects your pre-assumptions in the model i.e. the higher it is, the more you think you know what the answer should be, and the weights determine the strength of connection between this input and the output. These are the *hyper-parameters* that will be optimized.

* The activation function turns an unbounded input into a controllable, bounded input. A commonly used example would be a sigmoid function.

[1]: Neural Networks from Scratch - https://victorzhou.com

In [ ]:
import matplotlib.pyplot as plt 
import numpy as np 

x = np.linspace(-10, 10, 100) 
def sigmoid(x):
    return 1/(1 + np.exp(-x))

a = sigmoid(x)

plt.figure(figsize=(12,7)) # set the figsize
plt.plot(x, a) 
plt.xlabel("x") 
plt.ylabel("sigmoid(x)")
plt.rc('font',size=16) # set the font size
plt.show()

* We can then code a very simple neuron as follows:

In [ ]:
import numpy as np

class Neuron: # classes are a powerful aspect of python that allows inherited characteristics
  def __init__(self, weights, bias):
    self.weights = weights
    self.bias = bias

  def feedforward(self, inputs): # passing inputs to get an output is known as "feed forward"
    # Weight inputs, add bias, then use the activation function
    total = np.dot(self.weights, inputs) + self.bias
    return sigmoid(total)

weights = np.array([0, 1]) # w1 = 0, w2 = 1
bias = 4                   # b = 4
n = Neuron(weights, bias)

x = np.array([2, 3])       # x1 = 2, x2 = 3
print(n.feedforward(x))

* A neural network is just formed by combing neurons and connecting them.

![nn](images/network.svg)

* Again, we can make a simple code that implements this network.

In [ ]:
import numpy as np

class Neural_Network:
  '''
  A neural network with:
    - 2 inputs
    - a hidden layer with 2 neurons (h1, h2)
    - an output layer with 1 neuron (o1)
  Each neuron has the same weights and bias:
    - w = [0, 1]
    - b = 0
  '''
  def __init__(self):
    weights = np.array([0, 1])
    bias = 0

    # The Neuron class here is from the previous code sample
    self.h1 = Neuron(weights, bias)
    self.h2 = Neuron(weights, bias)
    self.o1 = Neuron(weights, bias)

  def feedforward(self, x):
    out_h1 = self.h1.feedforward(x)
    out_h2 = self.h2.feedforward(x)

    # The inputs for o1 are the outputs from h1 and h2
    out_o1 = self.o1.feedforward(np.array([out_h1, out_h2]))

    return out_o1

network = Neural_Network()
x = np.array([2, 3])
print(network.feedforward(x))

* This appears very abstract, but if we were classifying articles as either from physics or biology based on the number of equations in them, we might assign an output of (0) to physics and (1) to biology, and then train the network on data on equation numbers per article from each field. The training would attempt to minimize the *loss* in the output with respect to the parameters of the model.
* The number of hidden layers (the layers between input and output) determine the *depth* of the network and is the source of the term *deep learning*.

Convolutional neural networks
----------------------------------

* Nearly all image analysis relies on using Convolutional neural networks (CNN) rather than standard neural networks. This is for two main reasons:
    * Images are big - imagine building a neural network to process a 224x224 color image - including the 3 color channels (RGB) in the image, that comes out to 224 x 224 x 3 = 150 528 input features. A typical hidden layer in such a network might have 1024 nodes, so we’d have to train 150 528 x 1024 = 150+ million weights for the first layer alone. Our network would be huge and nearly impossible to train.
    * Positions can change - if you trained a network to detect dogs, you’d want it to be able to a detect a dog regardless of where it appears in the image. Imagine training a network that works well on a certain dog image, but then feeding it a slightly shifted version of the same image. The dog would not activate the same neurons, so the network would react completely differently.
* What we really want to do is learn characteristic features within our training images that would be general to the subject of interest rather than the image itself.

*Convolutions*

* A CNN basically convolves the image with a filter that will extract features. For example, the Sobel filter:

![sobel](images/vertical-sobel.svg)

when applied to a simple greyscale image, convolves it into a smaller matrix (using *padding* i.e. adding zeroes at the edges, would allow us to sample more pixels):

![convolve](images/convolve-output.gif)

at each step it is just performing an element-wise multiplication between the values in the filter and their corresponding values in the image, and then summing up all the element-wise products to get the new output.

* What does this actually do?

![lenna](images/lenna+vertical.png)

The Sobel filter is an edge detector - the image has been transformed into a set of *edge features*.

*Pooling*

* It is obvious to us (but not to the *machine*) that many features are connected such that many neighbouring pixels have similar values. In the context of the edge detection filter, if we find a strong edge at a location, it is pretty likely that the edge will also be present one pixel away.

* To avoid duplicating features, this is normally handled by using *pooling* of neighbouring pixels to there maximum, minimum or average (max-, min- and mean-pooling).

![pool](images/pool.gif)

*Softmax*

* Finally, we need an activation function, just as in the standard NN. A commonly used function for CNNs is Softmax, which has the following form:

${Softmax}(x_{i}) = \frac{\exp(x_i)}{\sum_j \exp(x_j)}$

* Softmax produces a value from 0 to 1, as for the Sigmoid, but also the outputs add up to 1, so it has the form of a probability - this is not needed, but it allows for the use of *cross entropy loss*, which includes a component of prediction confidence. So the very basic form of a CNN is the following:

![cnn](images/cnn.svg)

* Here we start with a 28x28 image and convolve it with 8 filters. Using a maximum pooling with size 2 reduces the image to 13x13. There are 10 Softmax nodes, which would assume we are trying to classify our data into 10 classes. The final output from Softmax for a given image would be the node with the highest probability i.e. the class prediction.

*Training*

* Before we actually let you play around with training your own network, a few comments on the process itself. It generally consists of two phases:
    * A forward phase, where the input is passed completely through the network - key values are stored in preparation for the backwards pass.
    * A backward phase, where gradients are backpropagated (backprop) and weights are updated. During this phase, each layer will receive the gradient of loss with respect to its outputs and return the gradient of loss with respect to its inputs - this establishes the dependence of loss on our parameters and allows for optimisation (with a standard optimiser like ADAM).
* The basic code for CNN training would have this form:

In [ ]:
# Feed forward
out = conv.forward(image)
out = pool.forward(out)
out = softmax.forward(out)

# Calculate initial gradient
gradient = np.zeros(10)
# ...

# Backprop
gradient = softmax.backprop(gradient)
gradient = pool.backprop(gradient)
gradient = conv.backprop(gradient)

* The core of training is in the `conv.backprop` step, as any changes in the filter weights is effectively changing the nature of the filters and the type of features associated with them.
* When really applying these methods in a scientific context, it is much more efficient to use highly optimised models, such as `tensorflow`. As in the tutorials.

Tutorials
=====================


1. Make sure you can run the simple examples in the lecture and understand how they build up into a *real* network.
2. Iris flower data set - a neural network trained on the properties of plants.
3. Number recognition - a convolutional neural network trained to recognize handwritten digits from 1-9.
4. CO functionalised AFM tip classification - a convolutional neural network trained to classify good and bad tips in AFM imaging.
5. Generative adversarial networks (GANs) - use a GAN to make realistic handwriting predictions.


## Iris flower data set

The Iris flower data set or Fisher's Iris data set is a multivariate data set introduced by the British statistician and biologist Ronald Fisher in his 1936 paper *"The use of multiple measurements in taxonomic problems as an example of linear discriminant analysis"*.

The data set consists of 50 samples from each of three species of Iris (Iris setosa, Iris virginica and Iris versicolor). Four features were measured from each sample: the length and the width of the sepals and petals, in centimeters.

We can import the needed libraries, and then we can load the iris data.

![iris](images/Iris_versicolor_3.jpeg)

In [ ]:
import pickle
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

with open("examples/data/iris.pkl", "rb") as fio:
    iris = pickle.load(fio)

In [ ]:
# lets see the data
from pprint import pprint
pprint(iris)

In [ ]:
# prepare input data
x = iris["data"]
input_shape = (4,)

In [ ]:
# prepare output data
target = iris["target"]
num_classes = 3
# one-hot representation
y = keras.utils.to_categorical(target, num_classes)

print(f"targets: {target[:5]}\none-hot rep:{y[:5, :]}")

In [ ]:
# shuffle
shuffle_indices = np.random.permutation(np.arange(len(target)))
x = x[shuffle_indices]
y = y[shuffle_indices]

In [ ]:
# test_train split 80-20
split = int(len(target) * 0.8)
x_train, x_test, y_train, y_test = x[:split, :], x[split:, :], y[:split, :], y[split:, :]

In [ ]:
print(input_shape)

In [ ]:
# model
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Dense(3, activation="relu"),
        layers.Dense(num_classes, activation="softmax")
    ]
)
model.summary()

In [ ]:
# train the network
batch_size = 5 # number of images in each batch used for optimisation - balance speed of each epoch against convergence to high accuracy
epochs = 200

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

In [ ]:
# accuracy
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

## Number recognition
Here is a CNN trained to recognize digits from images of handwritten numbers: 

![digits](images/digits.png)

In [ ]:
# get the data and scale it to convenient shapes
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets - test on data not in your training data!
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
# setup the model
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(), #matching array shapes
#        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

In [ ]:
# train the CNN
batch_size = 10000 # number of images in each batch used for optimisation - balance speed of each epoch against convergence to high accuracy
epochs = 3

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

In [ ]:
# accuracy
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

## CO functionalised AFM tip classification
An automated solution for carbon monoxide functionalization which combines machine learning descriptors with automated software control of the tip preparation process. 

![Schematic](images/CO-tip-evaluator.png)

[1] Alldritt, Benjamin, et al. "Automated tip functionalization via machine learning in scanning probe microscopy." Computer Physics Communications 273 (2022): 108258.

In [ ]:
# data ingestion
import numpy as np
from PIL import Image
import os

def add_norm_CO(X_):
    """ Normalise input image """
    sh = X_.shape

    for j in range(sh[0]):
        mean=np.mean(X_[j,:,:])            
        sigma=np.std(X_[j,:,:])          
        X_[j,:,:]-= mean
        X_[j,:,:]= X_[j,:,:]/ sigma

dataX = []
file_names = []
    
for filename in os.listdir("examples/images/tip_classification"):
    image_path = os.path.join("examples", "images", "tip_classification", filename)
    image = np.array((Image.open(image_path).resize((16, 16), Image.ANTIALIAS))).astype(np.float32)
    dataX.append(np.flipud(image))
    file_names.append(filename)
dataX = np.expand_dims(np.array(dataX), axis = 1)
add_norm_CO(dataX)
print (f'dataX: {dataX.shape}')

In [ ]:
# model
from tensorflow import keras
model = keras.models.load_model("examples/models/tip_classifier.h5")
model.summary()

In [ ]:
# prediction
outputs = model.predict_on_batch(dataX)
probs = np.squeeze(np.array(outputs))
preds = np.squeeze(np.round(outputs))

In [ ]:
# plotting
from matplotlib import cm
import matplotlib.pyplot as plt


def plot_preds_with_title(dataX,probs,file_names, cmap=cm.gray):
    cols = 10
    rows = -(-len(file_names) // cols)
    fig = plt.figure(figsize=(3.0*cols,3.0*rows))
    
    for i in range(len(file_names)):
        sp = fig.add_subplot(rows,cols,i+1)#, origin="lower"
        sp.imshow(dataX[i,0,:,:], cmap = cmap)   
        sp.axis('Off')
        desc = file_names[i].split("-")[0][:-1]
        sp.set_title(f'pred = {probs[i]: 0.3f}\ntarg = {desc} tip') 
    
    save_name = 'predicted.png'
    # plt.savefig('./'+save_name, bbox_inches='tight', dpi=200)
    plt.show()
    # plt.close()

plot_preds_with_title(dataX,probs,file_names)

## Generative adversarial networks (GANs)

* Using generative models we can learn the underlying distribution of a given dataset: after training, we can thus run these models in inference mode and generate new, realistic data points.

* An interesting category of generative models is given by Generative Adversarial Networks (https://arxiv.org/abs/1406.2661). 

* These architectures are based on a game theoretical approach, where two networks compete with each other and are simultaneously trained, until reaching an equilibrium.

### Generator

Let's suppose we want to generate images which could realistically belong to the MNIST handwritten dataset. A "Generator" network will be fed with vectors drawn from a random distribution, and its goal during the training will be to generate increasingly realistic images.


More specifically, the goal of the Generator will be maximixing the probability that *another* network, a "Discriminator" (which is trained to tell apart real from fake data) will classify its own, generated output as real.

![gen](images/gen.png)

In [ ]:
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import UpSampling2D, Conv2D
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# we will be generating 28x28 images
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)
latent_dim = 100

In [ ]:
generator = keras.Sequential()

generator.add(Dense(256, input_dim=latent_dim))
generator.add(LeakyReLU(alpha=0.2))
generator.add(BatchNormalization(momentum=0.8))
generator.add(Dense(512))
generator.add(LeakyReLU(alpha=0.2))
generator.add(BatchNormalization(momentum=0.8))
generator.add(Dense(1024))
generator.add(LeakyReLU(alpha=0.2))
generator.add(BatchNormalization(momentum=0.8))
generator.add(Dense(np.prod(img_shape), activation='tanh'))
generator.add(Reshape(img_shape))

generator.summary()

### Discriminator

The Discriminator network will receive as input both true (belonging to the dataset) and fake (output from the Generator) images, and will be trained to be as good as possible in classifying them correctly. 

The task will become more difficult with eath iteration, because at the same time the Generator is being trained to "trick" the Discriminator. 

![discr1](images/discr1.gif)
![discr2](images/discr2.gif)

In [ ]:
discriminator = keras.Sequential()

discriminator.add(Flatten(input_shape=img_shape))
discriminator.add(Dense(512))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(BatchNormalization(momentum=0.8))
discriminator.add(Dense(256))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dense(1, activation='sigmoid'))

discriminator.summary()

### Training

The architecture can be summarized as follows:

<img src="images/gan.png" width="850">

* What we expect during the training is that the Generator's loss will increase at the start, and then decrease, reaching a plateau. 

* Conversely, the Discriminator's loss will initially decrease almost to zero, and then gradually increase until plateuing as well. 

In [ ]:
# functions to plot losses and generated samples

def plot_loss(losses):
    d_loss = [v[0] for v in losses["D"]]
    g_loss = [v for v in losses["G"]]

    plt.figure(figsize=(10,8))
    plt.plot(d_loss, label="Discriminator loss")
    plt.plot(g_loss, label="Generator loss")
    plt.xlabel('Iterations')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()
    
def plot_generated(n_ex=10, dim=(1, 10), figsize=(12, 2)):
    noise = np.random.normal(0, 1, size=(n_ex, latent_dim))
    generated_images = generator.predict(noise)
    generated_images = generated_images.reshape(n_ex, 28, 28)

    plt.figure(figsize=figsize)
    for i in range(generated_images.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generated_images[i], interpolation='nearest', cmap='gray')
        plt.axis('off')
    plt.tight_layout()
    plt.show()

Let's compile the models to be trained

In [ ]:
# Adam optimizer
optimizer = Adam(0.0002, 0.5)

# compile the discriminator
discriminator.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])

# ---------------------
# Combined Model
# ---------------------

# While the discriminator can be trained indipendently,
# the generator needs to receive the discriminator's loss,
# to be able to adapt itself to it. 

# Thus we need to combine generator and discriminator in a single model,
# in which however the discriminator will be frozen (not trainable)

discriminator.trainable = False

# The generator takes noise as input and generates imgs
z = Input(shape=(latent_dim,))
img = generator(z)

# The discriminator takes generated images as input and determines validity
validity = discriminator(img)

# The combined model (stacked generator and discriminator) trains
# the generator to fool the discriminator
gan = Model(z, validity)
# compile the combined model
gan.compile(loss='binary_crossentropy', optimizer=optimizer) 

Load the dataset

In [ ]:
(X_train, _), (_, _) = mnist.load_data()
X_train = X_train / 127.5 - 1.   # Rescale from -1 to 1
X_train = np.expand_dims(X_train, axis=3)

Choose training hyperparameters

In [ ]:
# How many batches of data will we go through in the training
iterations = 15000
# Each with a size:
batch_size = 128
# How often we want to plot generated images and plot losses
plt_frq = 100

# A dictionary to store the losses
losses = {"D":[], "G":[]}

Training loop

In [ ]:
# Adversarial ground truths 
valid = np.ones((batch_size, 1)) 
fake = np.zeros((batch_size, 1))

for iteration in range(iterations):

    # ---------------------
    #  Train Discriminator
    # ---------------------

    # Select a random batch of images
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    imgs = X_train[idx]

    # Create noise vectors for the generator
    noise = np.random.normal(0, 1, (batch_size, latent_dim))

    # Generate a batch of new images
    gen_imgs = generator.predict(noise)

    # Train the discriminator
    discriminator.trainable = True
    
    # "train_on_batch" takes an input vector and a target vector: the real images are 
    # compared to the target outcome (a vector of 1s, i.e. all classified real), and viceversa 
    # for the generated images (compared to a vector of 0s i.e. all classified fake)
    d_loss_real = discriminator.train_on_batch(imgs, valid)
    d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
    
    # The final discriminator loss is given in equal parts by how good the discriminator is in 
    # correctly classifying both real and fake images
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # ---------------------
    #  Train Generator
    # ---------------------

    discriminator.trainable = False
    noise = np.random.normal(0, 1, (batch_size, latent_dim))

    # The generator loss decreases if the percentage of fake images classified as 
    # real by the discriminator increases
    g_loss = gan.train_on_batch(noise, valid)

    # Store losses 
    losses["D"].append(d_loss)
    losses["G"].append(g_loss)

    # Update the plots
    if iteration == 1 or iteration%plt_frq == 0:
        # Plot the progress
        print ("%d [D loss: %f] [G loss: %f]" % (iteration, d_loss[0], g_loss))
        plot_generated()
        plot_loss(losses)